In [1]:
# ch04: 线性回归
# 步骤1: 创建SparkSession对象
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lin_reg').getOrCreate()

In [2]:
# 步骤2: 读取数据集
df = spark.read.csv("Linear_regression_dataset.csv", inferSchema=True, header=True)

In [3]:
# 步骤3: 探究式数据分析
print(df.count(), len(df.columns))

1232 6


In [4]:
df.printSchema()

root
 |-- var_1: integer (nullable = true)
 |-- var_2: integer (nullable = true)
 |-- var_3: integer (nullable = true)
 |-- var_4: double (nullable = true)
 |-- var_5: double (nullable = true)
 |-- output: double (nullable = true)



In [5]:
df.describe().show(3, False)

+-------+-----------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|var_1            |var_2            |var_3             |var_4               |var_5               |output             |
+-------+-----------------+-----------------+------------------+--------------------+--------------------+-------------------+
|count  |1232             |1232             |1232              |1232                |1232                |1232               |
|mean   |715.0819805194806|715.0819805194806|80.90422077922078 |0.3263311688311693  |0.25927272727272715 |0.39734172077922014|
|stddev |91.5342940441652 |93.07993263118064|11.458139049993724|0.015012772334166148|0.012907228928000298|0.03326689862173776|
+-------+-----------------+-----------------+------------------+--------------------+--------------------+-------------------+
only showing top 3 rows



In [6]:
df.head(3)

[Row(var_1=734, var_2=688, var_3=81, var_4=0.328, var_5=0.259, output=0.418),
 Row(var_1=700, var_2=600, var_3=94, var_4=0.32, var_5=0.247, output=0.389),
 Row(var_1=712, var_2=705, var_3=93, var_4=0.311, var_5=0.247, output=0.417)]

In [7]:
from pyspark.sql.functions import corr
df.select(corr('var_1', 'output')).show()

+-------------------+
|corr(var_1, output)|
+-------------------+
| 0.9187399607627283|
+-------------------+



In [8]:
# 步骤4: 特征工程化
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
df.columns

['var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'output']

In [9]:
vec_assembler = VectorAssembler(inputCols=['var_1', 'var_2', 'var_3', 'var_4', 'var_5'], outputCol='features')

In [10]:
features_df = vec_assembler.transform(df)
features_df.printSchema()

root
 |-- var_1: integer (nullable = true)
 |-- var_2: integer (nullable = true)
 |-- var_3: integer (nullable = true)
 |-- var_4: double (nullable = true)
 |-- var_5: double (nullable = true)
 |-- output: double (nullable = true)
 |-- features: vector (nullable = true)



In [11]:
features_df.select('features').show(5, False)

+------------------------------+
|features                      |
+------------------------------+
|[734.0,688.0,81.0,0.328,0.259]|
|[700.0,600.0,94.0,0.32,0.247] |
|[712.0,705.0,93.0,0.311,0.247]|
|[734.0,806.0,69.0,0.315,0.26] |
|[613.0,759.0,61.0,0.302,0.24] |
+------------------------------+
only showing top 5 rows



In [12]:
model_df = features_df.select('features', 'output')

In [13]:
model_df.show(5, False)

+------------------------------+------+
|features                      |output|
+------------------------------+------+
|[734.0,688.0,81.0,0.328,0.259]|0.418 |
|[700.0,600.0,94.0,0.32,0.247] |0.389 |
|[712.0,705.0,93.0,0.311,0.247]|0.417 |
|[734.0,806.0,69.0,0.315,0.26] |0.415 |
|[613.0,759.0,61.0,0.302,0.24] |0.378 |
+------------------------------+------+
only showing top 5 rows



In [14]:
print(model_df.count(), len(model_df.columns))

1232 2


In [15]:
# 步骤5: 划分数据集
train_df, test_df = model_df.randomSplit([0.7, 0.3])
print((train_df.count(), len(train_df.columns)))

(875, 2)


In [16]:
print((test_df.count(), len(test_df.columns)))

(357, 2)


In [20]:
# 步骤6: 构建和训练线性回归模型
from pyspark.ml.regression import LinearRegression
lin_Reg = LinearRegression(labelCol='output')
lr_model = lin_Reg.fit(train_df)
print(lr_model.coefficients)

[0.00033045422117331137,5.0535719966951864e-05,0.00020349082704125253,-0.6236352275559593,0.53749365191754]


In [21]:
print(lr_model.intercept)

0.17248193161067485


In [22]:
train_predictions=lr_model.evaluate(train_df)

In [23]:
print(train_predictions.r2)

0.8689707188189623


In [ ]:
# 步骤7: 在测试数据集上评估线性回归模型
test_predictions=lr_model.evaluate(test_df)
print(test_predictions.r2)
pruint